# Introduction

---


This notebook details the complete workflow for building and evaluating a multi-label image classification model using the DenseNet201 architecture. The goal is to predict multiple labels simultaneously for each image within a dataset.  This notebook uses PyTorch for model building and training, scikit-learn for data splitting and some evaluation metrics.  The process is divided into several key steps, outlined below.

# Import Libraries

---


This section imports all the necessary Python libraries required for the project.  Key libraries include PyTorch (for deep learning, scikit-learn (for machine learning utilities), pandas (for data manipulation), and several others for image processing, visualization, and progress tracking.

In [ ]:
import torchvision
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from tqdm.auto import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
import seaborn as sns
import numpy as np
from torchvision import models, transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import itertools
from sklearn.metrics import classification_report
import random
import json
from plots import *
from helper import *
from metrics import *

In [1]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Files

---


This step loads the image data and corresponding labels. The image paths are read from a CSV file ('annotations.csv'), which contains filenames and their associated labels with associated feature presence.  The images the dataset used for this project is a collection of images of beehive frames with varying features. The images themselves are loaded from the 'images/raw/' directory.

In [ ]:
data_dir = 'data'
csv_path = os.path.join(data_dir, 'annotations.csv')
img_path = os.path.join(data_dir, 'images')
df = pd.read_csv(csv_path)

# Model Definition and Training

---


A custom dataset class ('**CustomImageDataset**') is defined to efficiently handle the image and label data during training. The pre-trained DenseNet201 model is loaded, and its final classification layer is adjusted to match the number of output labels (15 in this case). The model is then trained using binary cross-entropy loss ('**BCEWithLogitsLoss**') and an Adam optimizer. The training process is monitored and logged, with metrics recorded at each epoch.

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, data_transform=None):
        self.images = images
        self.labels = labels
        self.transform = data_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Apply transformers if provided
        if self.transform:
            image = self.transform(image)

        # Convert label from a list to a tensor
        label = torch.tensor(label, dtype=torch.float32)

        sample = {'Image': image, 'Label': label}

        return sample

In [ ]:
# Read in label file (annotations.csv)
df = pd.read_csv(csv_path)
#df = df[:5]


# Convert all labels to floats, NaN if blank
for col in df.columns[1:16]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop any images with NaNs in label columns
df.dropna(subset=df.columns[:16], inplace=True)
df.reset_index(drop=True, inplace=True)

# Extract ground truth labels for each image in label file (annotations.csv)
labels = []
for index, row in df.iterrows():
    labels.append(list(row.iloc[1:16]))

# Create path for and load in each image in directory
image_names = df['filename'].tolist()
images = []

for img_name in tqdm(image_names):
    image_path = os.path.join(img_path, img_name)
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    images.append(image)

# Split data into train (80%), validation (10%), and test (10%) sets
X_train_val, X_test, y_train_val, y_test = train_test_split(images, labels, test_size=1/10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=1/9, random_state=42)

In [ ]:
# define image transformations

densenet201_train_datatransform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

densenet201_test_datatransform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# Create datasets
train_data = CustomImageDataset(X_train, y_train, data_transform=densenet201_train_datatransform)
val_data = CustomImageDataset(X_val, y_val, data_transform=densenet201_test_datatransform)
test_data = CustomImageDataset(X_test, y_test, data_transform=densenet201_test_datatransform)

# Create data loaders
BATCH_SIZE = 32
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inherit the pre-trained weights from the DenseNet201 model
weights = models.DenseNet201_Weights.DEFAULT
model_d201 = models.densenet201(weights=weights).to(device)

# Freeze all layers (optional)
for param in model_d201.parameters():
    param.requires_grad = False

# Set output_shape variable to the number of labels in dataset
input_shape = 1920
output_shape = 15

# Recreate the `classifier` layer of the pre-trained model to custom output shape
model_d201.classifier = torch.nn.Sequential(
    torch.nn.Linear(input_shape,
                    output_shape),
).to(device)

# Training and Evaluation
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_d201.parameters(), lr=0.001)

densenet201_results = train(model=model_d201,
                      train_dataloader=train_dataloader,
                      test_dataloader=val_dataloader,
                      optimizer=optimizer,
                      loss_fn=criterion,
                      epochs=10,
                      device=device)

In [ ]:
# --- Testing on the Test Set ---
test_loss, test_ham_acc, test_zero_one_acc = test_step(model=model_d201,
                                                        dataloader=test_dataloader,
                                                        loss_fn=criterion,
                                                        device=device)
print(f"Test Loss: {test_loss:.4f}, Test Hamming Accuracy: {test_ham_acc:.4f}, Test Zero-One Accuracy: {test_zero_one_acc:.4f}")

# Hyperparmeter Tuning
---


To optimize performance, a hyperparameter search is conducted. Differnt combinations of batch size, number of epochs, and learning rate are tested, using both the original data transformations and a new transformation using 'transforms.TriviaAugmentWide'. The results of this search are saved to 'hyperparameter_testing_densenet201.csv' for analysis. The best performing hyperparameter set is then saved to 'best_params_d201'.

In [ ]:
densenet201_triv_aug_trans = transforms.Compose([
    transforms.Resize(size=(256, 256)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Trivial augment train data
train_data_triv_aug = CustomImageDataset(X_train, y_train, data_transform=densenet201_triv_aug_trans)

# Create different size batches of trivial aug train loaders
train_dataloader_triv_aug_16 = DataLoader(dataset=train_data_triv_aug,
                              batch_size=16,
                              shuffle=True)

train_dataloader_triv_aug_32 = DataLoader(dataset=train_data_triv_aug,
                              batch_size=32,
                              shuffle=True)

train_dataloader_triv_aug_64 = DataLoader(dataset=train_data_triv_aug,
                              batch_size=64,
                              shuffle=True)

In [ ]:
num_epochs = [5, 10]
lrs = [.001, .0001, .01]
batch_size = [16, 32, 64]

hyperparameter_combos = list(itertools.product(batch_size, num_epochs, lrs))
results = []

In [ ]:
#--------------------------------------Hyperparameter Testing--------------------------------------------------
# Create two datasets, one for each transform
train_data_triv_aug = CustomImageDataset(X_train, y_train, data_transform=densenet201_triv_aug_trans)
train_data_d201_transform = CustomImageDataset(X_train, y_train, data_transform=densenet201_train_datatransform)

for transform_name, train_data in [("triv_aug", train_data_triv_aug), ("d201_transform", train_data_d201_transform)]:
    for batch_size, epoch, lr in hyperparameter_combos:
        # Create DataLoader based on the current transform
        train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
        save_dir = f"runs/{transform_name}/lr_{lr}_batch_{batch_size}_epochs_{epoch}"
        os.makedirs(save_dir, exist_ok=True)
        writer = SummaryWriter(log_dir=save_dir)

        params = {'learning_rate': lr, 'batch_size': batch_size, 'epochs': epoch, 'transform': transform_name}
        writer.add_hparams(params, {})

        print(f"Starting training with hyperparameters: lr={lr}, batch_size={batch_size}, epochs={epoch}, transform={transform_name}")

        try:
            train_results = train(model=model_d201,
                                  train_dataloader=train_dataloader,
                                  test_dataloader=val_dataloader,
                                  optimizer=torch.optim.Adam(model_d201.parameters(), lr=lr),
                                  loss_fn=criterion,
                                  epochs=epoch,
                                  device=device,
                                  writer=writer,
                                  params=params)

            final_test_loss = train_results['test_loss'][-1]
            final_test_ham_acc = train_results['test_ham_acc'][-1]
            final_test_zero_one_acc = train_results['test_zero_one_acc'][-1]
            writer.add_hparams(params, {'Final Test Loss': final_test_loss, 'Final Test Hamming Acc': final_test_ham_acc, 'Final Test Zero-One Acc': final_test_zero_one_acc})

            results.append({'transform': transform_name, 'batch_size': batch_size, 'epochs': epoch, 'lr': lr, 'final_test_loss': final_test_loss, 'final_test_ham_acc': final_test_ham_acc, 'final_test_zero_one_acc': final_test_zero_one_acc})

        except Exception as e:
            print(f"An error occurred during training: {e}")
            results.append({'transform': transform_name, 'batch_size': batch_size, 'epochs': epoch, 'lr': lr, 'final_test_loss': float('nan'), 'final_test_ham_acc': float('nan'), 'final_test_zero_one_acc': float('nan')})

        finally:
            writer.close()

results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
#save result images
save_dir = os.getcwd()

In [ ]:
results_df = pd.DataFrame(results)
results_df['final_test_zero_one_acc'] = results_df['final_test_zero_one_acc'].apply(lambda x: x.item())
results_df.to_csv('hyperparameter_testing_densenet201.csv', index=False)

In [ ]:
best_row = results_df.loc[results_df['final_test_ham_acc'].idxmax()]

best_params = {
    'transform': best_row['transform'],
    'batch_size': best_row['batch_size'].item(),
    'epochs': best_row['epochs'].item(),
    'lr': best_row['lr'].item(),
    'final_test_loss': best_row['final_test_loss'].item(),
    'final_test_ham_acc': best_row['final_test_ham_acc'].item(),
    'final_test_zero_one_acc': best_row['final_test_zero_one_acc'].item()
}

filepath = 'best_params_d201.json'
with open(filepath, 'w') as f:
    json.dump(best_params, f, indent=4)
print(f"Best parameters saved to {filepath}")
print(best_params)

# Evaluations

---
After training, the model is evaluated on a held-out test set using several metrics. These include:
*   **Accuracy:** Overall correctness of predictions.
*   **Precision:** Proportion of correctly predicted positive labels among all predicted positive labels.
*   **Recall:** Proportion of correctly predicted positive labels among all actual positive labels.
*   **F1-Score:** Harmonic mean of precision and recall
*   **Confusion Matrix:** Visual represnetation of model predictions versus ground truth.
*   **ROC AUC:** Area under the ROC curve, measuring the model's ability to distinguish between classes.








In [ ]:
def how_did_i_do(model, test_dataloader, device):
    """Evaluates the model and computes metrics."""
    model.eval()
    y_true = []
    y_prob = []

    with torch.no_grad():
        for batch in test_dataloader:
            inputs = batch['Image'].to(device)
            labels = batch['Label'].to(device)
            outputs = model(inputs)

            y_true.extend(labels.cpu().numpy())
            probabilities = torch.sigmoid(outputs)
            y_prob.extend(probabilities.cpu().numpy())

    y_true = np.array(y_true)
    y_prob = np.array(y_prob)

    y_pred = (y_prob >= 0.5).astype(int)

    accuracy = compute_accuracy(y_true, y_pred)
    precision = compute_precision(y_true, y_pred, average='micro')
    recall = compute_recall(y_true, y_pred, average='micro')
    f1 = compute_f1_score(y_true, y_pred, average='micro')
    confusion_mat = compute_confusion_matrix(y_true.flatten(), y_pred.flatten())
    roc_auc = compute_roc_auc(y_true.flatten(), y_prob.flatten())
    fpr, tpr, roc_auc_value = compute_roc_curve_data(y_true.flatten(), y_prob.flatten())



    # Print the metrics
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"Confusion Matrix:\n{confusion_mat}")
    print(f"ROC AUC: {roc_auc}")
    print(f"FPR: {fpr}")
    print(f"TPR: {tpr}")
    print(f"ROC AUC Value: {roc_auc_value}")

    return accuracy, precision, recall, f1, confusion_mat, roc_auc, fpr, tpr, roc_auc_value


In [ ]:
accuracy, precision, recall, f1, confusion_mat, roc_auc, fpr, tpr, roc_auc_value = how_did_i_do(model_d201, test_dataloader, device)

# Visualizations

---
Finally, visualizations are generated to aid in the understanding of the model's performance. A confusion matrix is plotted to show the distribution of correct and incorre t predictions. Plots comparing training and validation loss and accuracy over epochs are generated as well as visualizations for the AUC.


### Confusion Matrix

In [ ]:
plot_confusion_matrix(model_d201, test_dataloader, device, 'd201', save_dir)

### Training and Validation Comparisons per Metric

In [ ]:
plot_training_validation_metrics(save_dir, train_results, 'd201')

### Area Under the Curve (AUC)

In [ ]:
plot_roc_curves(save_dir, model_d201, test_dataloader, device, 'd201')

# References

---


1.   **Gallery of transformations:**<br>
      PyTorch.org. "Plot Transforms Illustrations." PyTorch, https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py. Accessed 01 November 2024.
2.   **DataLoaders:**<br>
      PyTorch.org. "torch.utils.data.DataLoader — PyTorch 2.1.0+cu118 documentation." PyTorch, https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader. Accessed 03 November 2024.
3.    **PyTorch Deep Learning (Train and Test Loops):**<br>
      Bourke, Daniel. "engine.py." pytorch-deep-learning, https://github.com/mrdbourke/pytorch-deep-learning/blob/main/going_modular/going_modular/engine.py. Accessed 03 November 2024.
4.    **PyTorch Torchvision Blog Post:**<br>
      PyTorch Blog. "How to Train State-of-the-Art Models Using Torchvision’s Latest Primitives." PyTorch, https://pytorch.org/blog/how-to-train-state-of-the-art-models-using-torchvision-latest-primitives/. Accessed 03 November 2024.
5.    **DenseNet201 Tutorial:**<br>
      Yesil Science. "Transfer Learning DenseNet201." Yesil Science, https://yesilscience.com/transfer-learning-densenet201/. Accessed 20 November 2024.







